In [2]:
import numpy as np
from PIL import Image
from scipy.optimize import brentq
from stl import mesh

def load_and_resize_image(image_path, width_mm, height_mm, min_feature_size_mm):
    # Load image
    img = Image.open(image_path)

    # Calculate the required number of pixels based on minimum feature size
    num_pixels_x = int(width_mm / min_feature_size_mm)
    num_pixels_y = int(height_mm / min_feature_size_mm)

    # Resize image to desired dimensions
    img_resized = img.resize((num_pixels_x, num_pixels_y), Image.LANCZOS)

    return img_resized

def convert_to_grayscale(img):
    # Convert to numpy array
    img_np = np.array(img)

    # Calculate luminance using human eye adjusted formula
    if img_np.ndim == 3:
        R = img_np[:, :, 0]
        G = img_np[:, :, 1]
        B = img_np[:, :, 2]
        L = 0.2126 * R + 0.7152 * G + 0.0722 * B
    else:
        L = img_np

    L = np.clip(L, 0, 255)
    return L

def maximize_contrast(L):
    min_val = np.min(L)
    max_val = np.max(L)
    if max_val - min_val == 0:
        return L

    L_stretched = (L - min_val) * 255.0 / (max_val - min_val)
    return L_stretched

def calculate_desired_transmittance(L, epsilon_R, epsilon_G, epsilon_B, l_min, l_max):
    T_R = 2.71828 ** (-epsilon_R * l_min)
    T_G = 2.71828 ** (-epsilon_G * l_min)
    T_B = 2.71828 ** (-epsilon_B * l_min)
    T_max = 0.2126 * T_R + 0.7152 * T_G + 0.0722 * T_B
    T_R = 2.71828 ** (-epsilon_R * l_max)
    T_G = 2.71828 ** (-epsilon_G * l_max)
    T_B = 2.71828 ** (-epsilon_B * l_max)
    T_min = 0.2126 * T_R + 0.7152 * T_G + 0.0722 * T_B    
    
    T_desired = ((L / 255.0) * (T_max - T_min)) + T_min
    return T_desired

def solve_for_thickness(T_desired, epsilon_R, epsilon_G, epsilon_B, l_min, l_max):
    thickness = np.zeros_like(T_desired)
    T_desired_flat = T_desired.flatten()
    thickness_flat = thickness.flatten()

    def f(l, T_desired_pixel):
        T_R = 2.71828 ** (-epsilon_R * l)
        T_G = 2.71828 ** (-epsilon_G * l)
        T_B = 2.71828 ** (-epsilon_B * l)
        T = 0.2126 * T_R + 0.7152 * T_G + 0.0722 * T_B
        return T - T_desired_pixel

    for i, T_desired_pixel in enumerate(T_desired_flat):
        if T_desired_pixel <= 0:
            thickness_flat[i] = l_max
            continue
        elif T_desired_pixel >= 1:
            thickness_flat[i] = l_min
            continue

        try:
            l_solution = brentq(f, l_min, l_max, args=(T_desired_pixel,))
            thickness_flat[i] = l_solution
        except ValueError:
            f_l_min = f(l_min, T_desired_pixel)
            f_l_max = f(l_max, T_desired_pixel)
            thickness_flat[i] = l_min if abs(f_l_min) < abs(f_l_max) else l_max

    thickness = thickness_flat.reshape(T_desired.shape)
    return thickness

def quantize_thickness(thickness, l_min, l_max, delta_l):
    quantized_thickness = np.round((thickness - l_min) / delta_l) * delta_l + l_min
    quantized_thickness = np.clip(quantized_thickness, l_min, l_max)
    return quantized_thickness

def apply_dithering(thickness, l_min, l_max, delta_l):
    height, width = thickness.shape
    thickness_dithered = thickness.copy()
    pad_width = 1
    padded_thickness = np.pad(thickness_dithered, pad_width, mode='constant', constant_values=0)

    for y in range(height):
        for x in range(width):
            old_thickness = padded_thickness[y + pad_width, x + pad_width]
            new_thickness = np.round((old_thickness - l_min) / delta_l) * delta_l + l_min
            new_thickness = np.clip(new_thickness, l_min, l_max)
            padded_thickness[y + pad_width, x + pad_width] = new_thickness
            quant_error = old_thickness - new_thickness
            if x + 1 < width:
                padded_thickness[y + pad_width, x + pad_width + 1] += quant_error * 7 / 16
            if x - 1 >= 0 and y + 1 < height:
                padded_thickness[y + pad_width + 1, x + pad_width - 1] += quant_error * 3 / 16
            if y + 1 < height:
                padded_thickness[y + pad_width + 1, x + pad_width] += quant_error * 5 / 16
            if x + 1 < width and y + 1 < height:
                padded_thickness[y + pad_width + 1, x + pad_width + 1] += quant_error * 1 / 16

    final_thickness = padded_thickness[pad_width:-pad_width, pad_width:-pad_width]
    return final_thickness

def generate_heightmap_image(thickness, l_min, l_max):
    heightmap = (thickness - l_min) * 255.0 / (l_max - l_min)
    heightmap = np.clip(heightmap, 0, 255).astype(np.uint8)
    heightmap = 255 - heightmap  # Invert for visualization
    heightmap_img = Image.fromarray(heightmap)
    return heightmap_img

def generate_stl(thickness, width_mm, height_mm, min_feature_size_mm, output_filename):
    height_px, width_px = thickness.shape

    # Calculate pixel dimensions
    pixel_width = width_mm / width_px
    pixel_height = height_mm / height_px

    vertices = []
    faces = []
    vertex_index = 0

    for i in range(height_px):
        for j in range(width_px):
            # Coordinates of pixel corners
            x0 = j * pixel_width
            x1 = (j + 1) * pixel_width
            y0 = i * pixel_height
            y1 = (i + 1) * pixel_height
            z = thickness[i, j]

            # Create 8 vertices per pixel
            # Bottom vertices (z=0)
            v0 = [x0, y0, 0]
            v1 = [x1, y0, 0]
            v2 = [x1, y1, 0]
            v3 = [x0, y1, 0]

            # Top vertices (z=thickness)
            v4 = [x0, y0, z]
            v5 = [x1, y0, z]
            v6 = [x1, y1, z]
            v7 = [x0, y1, z]

            # Add vertices
            vertices.extend([v0, v1, v2, v3, v4, v5, v6, v7])

            # Create faces (using indices of the vertices)
            # Top face (v4, v5, v6, v7)
            faces.append([vertex_index+4, vertex_index+5, vertex_index+6])
            faces.append([vertex_index+4, vertex_index+6, vertex_index+7])

            # Bottom face (v0, v1, v2, v3) - reverse order for normals
            faces.append([vertex_index+0, vertex_index+2, vertex_index+1])
            faces.append([vertex_index+0, vertex_index+3, vertex_index+2])

            # Side faces
            # Side 1 (v0, v1, v5, v4)
            faces.append([vertex_index+0, vertex_index+1, vertex_index+5])
            faces.append([vertex_index+0, vertex_index+5, vertex_index+4])

            # Side 2 (v1, v2, v6, v5)
            faces.append([vertex_index+1, vertex_index+2, vertex_index+6])
            faces.append([vertex_index+1, vertex_index+6, vertex_index+5])

            # Side 3 (v2, v3, v7, v6)
            faces.append([vertex_index+2, vertex_index+3, vertex_index+7])
            faces.append([vertex_index+2, vertex_index+7, vertex_index+6])

            # Side 4 (v3, v0, v4, v7)
            faces.append([vertex_index+3, vertex_index+0, vertex_index+4])
            faces.append([vertex_index+3, vertex_index+4, vertex_index+7])

            vertex_index += 8

    # Convert lists to numpy arrays
    vertices = np.array(vertices)
    faces = np.array(faces)

    # Create mesh
    lithophane_mesh = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
    for i, f in enumerate(faces):
        for j in range(3):
            lithophane_mesh.vectors[i][j] = vertices[f[j], :]

    lithophane_mesh.save(output_filename)    

In [5]:
# Input parameters
filename = '3ElephantsCropped'
image_path = 'InputImages/'+filename+'.JPG'
#Absorbance values of the filament being used
#Values provided are for white PLA (Sunlo brand)
epsilon_R = 0.42  #Provide actual absorbance values
epsilon_G = 0.54
epsilon_B = 0.73
#Minimum and maximum thickness
l_min = 0.4
l_max = 4.2
#Layer height
delta_l = 0.2
#Final lithophane dimensions
width_mm = 170
height_mm = 118
#Remove features below this size
min_feature_size_mm = 0.3
output_stl_filename = filename+'.stl'
output_heightmap_filename = filename+'.png'

# Step 2: Load and resize the image
img_resized = load_and_resize_image(image_path, width_mm, height_mm, min_feature_size_mm)

# Step 3: Convert to grayscale
L = convert_to_grayscale(img_resized)

# Step 4: Maximize contrast
L_stretched = maximize_contrast(L)

# Step 5: Calculate desired transmittance
T_desired = calculate_desired_transmittance(L_stretched, epsilon_R, epsilon_G, epsilon_B, l_min, l_max)

# Step 6: Calculate required thickness
thickness = solve_for_thickness(T_desired, epsilon_R, epsilon_G, epsilon_B, l_min, l_max)

# Step 7: Quantize thickness values
quantized_thickness = quantize_thickness(thickness, l_min, l_max, delta_l)

# Step 8: Apply dithering
dithered_thickness = apply_dithering(quantized_thickness, l_min, l_max, delta_l)

# Step 9: Generate heightmap image
heightmap_img = generate_heightmap_image(dithered_thickness, l_min, l_max)
heightmap_img.save(output_heightmap_filename)

# Step 10: Generate .stl file
generate_stl(dithered_thickness, width_mm, height_mm, min_feature_size_mm, output_stl_filename)

print('Lithophane .stl file generated:', output_stl_filename)
print('Heightmap image saved as:', output_heightmap_filename)

Lithophane .stl file generated: 3ElephantsCropped.stl
Heightmap image saved as: 3ElephantsCropped.png
